<a href="https://colab.research.google.com/github/jeduardosleite/sql_basic_2/blob/main/SQL_exercicio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving query_1.csv to query_1.csv
Saving query_2.csv to query_2.csv
Saving query_3.csv to query_3.csv
Saving query_4.csv to query_4.csv
Saving query_5.csv to query_5.csv
Saving query_6.csv to query_6.csv
Saving query_7.csv to query_7.csv


In [9]:
# Creating the variables for demonstration.

query_1 = pd.read_csv("query_1.csv")
query_2 = pd.read_csv("query_2.csv")
query_3 = pd.read_csv("query_3.csv")
query_4 = pd.read_csv("query_4.csv")
query_5 = pd.read_csv("query_5.csv")
query_6 = pd.read_csv("query_6.csv")
query_7 = pd.read_csv("query_7.csv")

# **Module 21 - Introduction SQL II**

I created a table using the code:
```sql
CREATE EXTERNAL TABLE clientes(
    id BIGINT,
    idade BIGINT,
    sexo STRING,
    dependentes BIGINT,
    escolaridade STRING,
    tipo_cartao STRING,
    limite_credito DOUBLE,
    valor_transacao_12m DOUBLE,
    qtd_transacao_12m BIGINT
)
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'separatorChar' = ',',
  'quoteChar' = '"',
  'escapeChar' = '\\'
)
STORED AS TEXTFILE
LOCATION 's3://jeduardo-m2-exercicio/';
```

In this exercise i learned:
- condition *WHERE*
- insert new data
- work with partitioned
- add new columns
- drop a table

### **Query 1**
```sql
SELECT id, idade, sexo, dependentes
FROM clientes;
```

In [10]:
query_1

,id,idade,sexo,dependentes
0,768805383,45,M,3
1,818770008,49,F,5
2,713982108,51,M,3


### **Query 2**
```sql
SELECT id, valor_transacao_12m
FROM clientes
WHERE escolaridade = 'mestrado' AND sexo = 'F';
```

In [11]:
query_2

,id,valor_transacao_12m
0,818770008,1291.45


### **Query 3**
```sql
SELECT sexo, AVG(idade)
AS "media_idade_por_sexo"
FROM clientes
GROUP BY sexo;
```


In [12]:
query_3

,sexo,media_idade_por_sexo
0,F,49.0
1,M,48.0


### **Query 4**
```sql
SELECT * FROM clientes;
```

In [13]:
query_4

,id,idade,sexo,dependentes,escolaridade,tipo_cartao,limite_credito,valor_transacao_12m,qtd_transacao_12m
0,768805383,45,M,3,ensino medio,blue,12691.51,1144.90,42
1,818770008,49,F,5,mestrado,solteiro,8256.96,1291.45,33
2,713982108,51,M,3,mestrado,casado,3418.56,1887.72,20
3,768805451,56,M,6,ensino medio,casado,7591.51,1564.90,50
4,456556,30,F,0,mestrado,solteiro,5526.96,55251.45,10
5,545,49,F,2,doutorado,blue,3417.55,555.72,7


## **WORKING WITH PARTITIONED**

Its a technique for dividing a large table into small, organized parts.

Each partition has subsets of data.<br>
<br>
1) Created a bucket in *AWS S3*.

2) Created a table
```sql
CREATE EXTERNAL TABLE clientes_part(
    id BIGINT,
    idade BIGINT,
    dependentes BIGINT,
    escolaridade STRING,
    tipo_cartao STRING,
    limite_credito DOUBLE,
    valor_transacao_12m DOUBLE,
    qtd_transacao_12m BIGINT
)
PARTITIONED BY (sexo STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES ('separatorChar' = ',', 'quoteChar' = '"', 'escapeChar' = '\\')
STORED AS TEXTFILE
LOCATION 's3://bucket-partitioned-jeduardo/'
```
3) Inform the *AWS Athena*
```sql
MSCK REPAIR TABLE clientes_part
```

### **Query 5**

```sql
SELECT * FROM clientes_part
WHERE sexo = 'F';
```

In [14]:
query_5

,id,idade,dependentes,escolaridade,tipo_cartao,limite_credito,valor_transacao_12m,qtd_transacao_12m,sexo
0,456556,30,0,mestrado,solteiro,5526.96,55251.45,10,F
1,545,49,2,doutorado,blue,3417.55,555.72,7,F
2,818770008,49,5,mestrado,solteiro,8256.96,1291.45,33,F


### **Query 6**

```sql
SELECT id, idade, limite_credito
FROM clientes
WHERE sexo = 'M'
ORDER BY limite_credito DESC;
```

In [15]:
query_6

,id,idade,limite_credito
0,768805383,45,12691.51
1,768805451,56,7591.51
2,713982108,51,3418.56


### **Query 7**

```sql
ALTER TABLE clientes
ADD COLUMNS (estado STRING)
WHERE sexo = 'F';
```

In [17]:
query_7

,id,idade,sexo,dependentes,escolaridade,tipo_cartao,limite_credito,valor_transacao_12m,qtd_transacao_12m,estado
0,768805383,45,M,3,ensino medio,blue,12691.51,1144.90,42,NaN
1,818770008,49,F,5,mestrado,solteiro,8256.96,1291.45,33,NaN
2,713982108,51,M,3,mestrado,casado,3418.56,1887.72,20,NaN
3,768805451,56,M,6,ensino medio,casado,7591.51,1564.90,50,NaN
4,456556,30,F,0,mestrado,solteiro,5526.96,55251.45,10,NaN
5,545,49,F,2,doutorado,blue,3417.55,555.72,7,NaN


In the end, i drop this table using:
```sql
DROP TABLE clientes_part;
```